## Week 7 Lab Manual
### Foundations of Deep Learning & AI Functionality

**Instructor Note**: This lab manual provides the aim, code, and explanation for each practical task. Focus on the architectural patterns and the transition from theoretical concepts to functional AI implementations.

---

# Week 7: Advanced RAG & Reasoning
## Solving Complex Queries with Gemini 2.5 Flash

###  Weekly Table of Contents
1. [Multi-Query Retrieval & Gemini Re-Ranking](#-Lab-7.1:-Multi-Query-Retrieval-&-Gemini-Re-Ranking)
2. [Building a Self-Reflective Agent with LangGraph](#-Lab-7.2:-Building-a-Self-Reflective-Agent-with-LangGraph-(Ollama-+-Gemini))
- Multi-Perspective Query Expansion
- Stateful Graph Workflows
- Agentic Self-Correction

###  Learning Objectives
Basic RAG finds text; Advanced RAG finds *answers*. This week we master:
1.  **Multi-Query Retrieval**: Using Gemini to rewrite user queries into multiple variations.
2.  **Contextual Compression**: Filtering out "noise" from retrieved documents.
3.  **Reranking**: Using a Rerank model (or Gemini's reasoning) to sort context by relevance.
4.  **Long-Context RAG**: Leveraging Gemini's 1.5M token window for "needle-in-a-haystack" tasks.

---

###  7.1 Deep Learning: Reranking & Compression

#### The Retrieval Gap
Sometimes semantic search finds a document that is *semantically* similar but *factually* irrelevant.
*   **The Solution**: A "Reranker" acts as a second stage. It looks at the Top 20 results from semantic search and carefully scores them against the question.

#### Contextual Compression
LLMs have a context window, but filling it with 10,000 words of "maybe relevant" text can confuse the model. We use compression to extract only the sentences that actually help answer the question.

---

In [ ]:
# 📦 WEEK 7 INITIALIZATION
import os
from dotenv import load_dotenv
import google.generativeai as genai
from IPython.display import Markdown, display

# LANGCHAIN & LANGGRAPH
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langgraph.graph import END, StateGraph
from typing import List, TypedDict

# --- CONFIGURATION ---
load_dotenv(override=True)
MODEL = "gemini-1.5-flash"

# Ensure API Key is available
GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY") or os.getenv("GEMINI_API_KEY")
if GEMINI_API_KEY:
    os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
    genai.configure(api_key=GEMINI_API_KEY)
else:
    print("⚠️ WARNING: GEMINI_API_KEY not found.")

# Cloud Model Instance
llm_cloud = ChatGoogleGenerativeAI(model=MODEL, temperature=0)

print(f"✅ Week 7 Ready: Advanced RAG with {MODEL}")


##  Lab 7.1: Multi-Query Retrieval & Gemini Re-Ranking
**Aim**: To overcome the limitations of standard distance-based similarity search by expanding a single user query into multiple perspectives and re-ranking the results for maximum precision.

**Explanation**:
This lab implements two critical "Advanced RAG" patterns:
1.  **Query Expansion**: We use an LLM to generate variations of the user's question. This ensures that even if the user uses non-technical language, at least one variation will likely match the technical index.
2.  **Re-Ranking**: We take raw results and apply a second "reasoning" pass (using Gemini 1.5 Flash's massive context window) to ensure factual alignment before the final answer generation.

In [ ]:
# 1. MULTI-QUERY RETRIEVAL (Query Expansion)
# We use Gemini to rewrite the question into 5 variations to catch different semantic angles.

multi_query_prompt = ChatPromptTemplate.from_template("""
You are an AI assistant tasked with generating five different versions of the given user 
question to retrieve relevant documents from a vector database. My goal is to overcome 
limitations of standard distance-based search.

Original question: {question}

Provide 5 variations, one per line:""")

generate_queries = (
    multi_query_prompt 
    | llm_cloud 
    | StrOutputParser() 
    | (lambda x: [q.strip() for q in x.split("\n") if q.strip()])
)

# Test query expansion
test_question = "What is the impact of salt on climate change?"
# queries = generate_queries.invoke({"question": test_question})
# print("--- Generated Variations ---")
# for i, q in enumerate(queries): print(f"{i+1}: {q}")


In [ ]:
# 2. GEMINI RE-RANKER
# Once we have results, we use Gemini's reasoning to pick the most factually aligned one.

rerank_prompt = ChatPromptTemplate.from_template("""
You are an expert re-ranker. Given the following user question and a list of retrieved documents, 
rank the documents from most relevant to least relevant. Return only the index numbers in order of relevance.

Question: {question}

Documents (Index: Content):
{documents}

Ranked Indices (comma separated):""")

reranker_chain = rerank_prompt | llm_cloud | StrOutputParser()

# Simulated retrieval results
docs = [
    "0: Salt mining in the Himalayas has increased recently.",
    "1: Increased ocean salinity can affect thermohaline circulation and global climate patterns.",
    "2: Salt is commonly used for de-icing roads in winter.",
    "3: Soil salinization is a major threat to global food security but is indirectly linked to climate change."
]

# result = reranker_chain.invoke({"question": test_question, "documents": "\n".join(docs)})
# print(f"Ranked Indices: {result}")


## 1. Beyond Basic RAG: The Need for Advanced Orchestration

Basic RAG (Retrieval-Augmented Generation) often fails when:
1.  **Poor Query Formulation:** The user's question doesn't match the technical language of the documents.
2.  **Low Precision Retrieval:** The top-k results contain irrelevant noise that confuses the LLM.
3.  **Complex Reasoning:** The answer requires synthesizing information from multiple distant parts of the dataset.

### Advanced Techniques:
*   **Multi-Query Retrieval:** Using an LLM to generate multiple versions of a query to capture different semantic angles.
*   **Re-Ranking:** Retrieving a large set of documents (e.g., top 20) and using a more powerful model (or cross-encoder) to pick the most relevant 3.
*   **Cyclic Workflows (Agents):** Instead of a linear pipeline, we use a loop where the model can "reflect" on whether the retrieved information is sufficient.

---

## 2. Introducing LangGraph

While LangChain is great for chains, **LangGraph** allows us to build stateful, multi-actor applications with loops. It is the core framework for building robust AI Agents.

### Core Concepts:
1.  **State:** A shared object that evolves as nodes execute.
2.  **Nodes:** Python functions that take the state and return an update.
3.  **Edges:** Define the flow between nodes (can be conditional).

##  Lab 7.2: Building a Self-Reflective Agent with LangGraph
**Aim**: To transition from linear RAG chains to cyclic, stateful agentic workflows that can "reflect" on the quality of retrieved data and re-try queries if the initial results are insufficient.

**Explanation**:
This represents the cutting edge of LLM application development:
1.  **LangGraph Orchestration**: We define a "State" that tracks the conversation and a "Graph" that defines the logic flow (e.g., Search -> Evaluate -> Answer).
2.  **Self-Correction**: The agent includes a "Node" that checks if the retrieved context actually answers the question. If not, it loops back to perform a broader search.
3.  **Hybrid Reasoning**: Uses Gemini 1.5 Flash for the complex planning/reflection steps.

In [ ]:
# 🧪 Lab 7.2: SELF-REFLECTIVE AGENT
# Transitioning from linear chains to cycles.

# 1. Define the Graph State
class GraphState(TypedDict):
    question: str
    generation: str
    documents: List[str]

# 2. Define the Graph Nodes (Logic Units)
def retrieve(state: GraphState):
    print("---[RETRIEVING]---")
    # Simulate DB search
    return {"documents": ["Salt concentration affects ocean density."], "question": state["question"]}

def grade_documents(state: GraphState):
    print("---[GRADING]---")
    # In a full app, we'd use llm_cloud to check if the doc actually answers the question
    if "Salt" in state["documents"][0]:
        return "generate"
    else:
        return "rewrite"

def generate(state: GraphState):
    print("---[GENERATING]---")
    # Use llm_cloud to generate final response based on docs
    response = llm_cloud.invoke(f"Use these docs: {state['documents']} to answer: {state['question']}")
    return {"generation": response.content}

# 3. Assemble the StateGraph
workflow = StateGraph(GraphState)

# Add Nodes
workflow.add_node("retrieve", retrieve)
workflow.add_node("generate", generate)

# Define Logic Flow
workflow.set_entry_point("retrieve")

# Add conditional routing from 'retrieve' node
workflow.add_conditional_edges(
    "retrieve",
    grade_documents,
    {
        "generate": "generate",
        "rewrite": "retrieve" # Loop back if info is missing
    }
)

workflow.add_edge("generate", END)

# 4. Compile and Run
app = workflow.compile()

# Test Execution
inputs = {"question": "How does salt affect climate?"}
print("--- STARTING LANGGRAPH EXECUTION ---")
for output in app.stream(inputs):
    for key, value in output.items():
        print(f"Node completed: {key}")
        if "generation" in value:
            print(f"\nFinal Answer: {value['generation']}")


---

##  Instructor's Evaluation & Lab Summary

###  Assessment Criteria
1. **Technical Implementation**: Adherence to the lab objectives and code functionality.
2. **Logic & Reasoning**: Clarity in the explanation of the underlying AI principles.
3. **Best Practices**: Use of secure environment variables and structured prompts.

**Lab Completion Status: Verified**
**Focus Area**: Language Modelling & Deep Learning Systems.